In [ ]:
import torch
import torch.nn as nn
import time


# try to write simple liner by myself



## prepare data with training dataset

In [ ]:
# 设置随机种子以确保结果可复现
torch.manual_seed(42)

X = torch.rand(1000,4) # 4 featcure 100 rows Upper case X means traning dataset
y = torch.ones(1000,1) *5 # label data

y # y=f(x)+b 最终每一行的数据都要到5这里



In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = nn.Linear(4, 16);
        self.layer2 = nn.Linear(16,1);
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x) 
        #x = self.relu(x) # do wen need this?
        x
        return x

model = MyNet()

In [ ]:
def probeParam():
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total number of parameters: {total_params}")
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Size: {param.size()} | Parameters: {param.numel()} | {param}")
    #grad
    for name, param in model.named_parameters():
        if param.grad is not None:
            print(f"  {name} gradient: {param.grad}")


In [ ]:
probeParam()

## 定义损失函数

In [ ]:
loss_fn = nn.MSELoss()
loss_fn

## 定义优化器

In [ ]:
import torch.optim as optim

optimizier = optim.Adam(model.parameters(), lr=0.01) # 定义优化器和学习率
optimizier

In [ ]:
losses = []
epoch_times = []

def train_in_epoch(epoch_index=1):

    start_time = time.time()  # 记录开始时间
    
    outputs = model(X)
    #print(outputs)
    #print(y.shape)
    loss = loss_fn(outputs , y) #计算结果和标注数据的 差 
    optimizier.zero_grad() 
    loss.backward()
    optimizier.step()
    
    # 记录损失
    losses.append(loss.item())
    end_time = time.time()  # 记录结束时间
    epoch_time = end_time - start_time
    epoch_times.append(epoch_time)
    
    # 打印每100个epoch的损失和时间
    if (epoch_index+1) % 100 == 0:
        print(f'Epoch [{epoch_index+1}/{num_epochs}], Loss: {loss.item():.4f}, Time: {epoch_time:.4f} seconds')
        


In [ ]:
# 训练模型
num_epochs = 1000
for epoch_index in range(num_epochs):
    train_in_epoch(epoch_index)

In [ ]:
num_epochs, len(losses),len(epoch_times)

In [ ]:
import matplotlib.pyplot as plt
def showtime():
    # 计算平均epoch时间
    
   
    
    avg_epoch_time = sum(epoch_times) / len(epoch_times)
    print(f"\nAverage epoch time: {avg_epoch_time:.4f} seconds")
    
    
    # 绘制损失曲线
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs+1), losses)
    plt.title('Training Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # 绘制epoch时间曲线
    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs+1), epoch_times)
    plt.title('Epoch Time Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Time (seconds)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
%matplotlib inline
showtime()

In [ ]:
## checkout model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch

# 测试模型
model.eval()
with torch.no_grad():
    test_output = model(X)

# 将张量转换为 NumPy 数组以便绘图
predicted = test_output.numpy().flatten()
actual = y.numpy().flatten()

# 创建 x 轴的值（样本索引）
x = range(len(predicted))

# 创建图表
plt.figure(figsize=(12, 6))
plt.plot(x, predicted, label='Predicted', marker='o', linestyle='-', markersize=8)
plt.plot(x, actual, label='Actual', marker='s', linestyle='--', markersize=8)

# 添加标题和标签
plt.title('Predicted vs Actual Values', fontsize=16)
plt.xlabel('Sample Index', fontsize=12)
plt.ylabel('Value', fontsize=12)

# 添加图例
plt.legend(fontsize=12)

# 显示网格
plt.grid(True, linestyle=':', alpha=0.7)

# 调整 y 轴的范围以更好地显示数据
y_min = min(min(predicted), min(actual))
y_max = max(max(predicted), max(actual))
plt.ylim(y_min - 0.5, y_max + 0.5)

# 显示图表
plt.show()

# 打印数值
#print("预测值:")
#print(test_output)
#print("\n实际值:")
#print(y)
#print("\n差值:")
#print(test_output - y)

## 切换成GPU版本

如何安装CUDA之类的就单说了



In [ ]:
import torch

# 检查 CUDA 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 设置随机种子以确保结果可复现
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
# 生成数据并移到 GPU
X = torch.rand(1000, 4, device=device)  # 4 features, 1000 rows
y = torch.ones(1000, 1, device=device) * 5  # label data
y # TODO 这个时候在内存和GPU中都有数据吗？

In [ ]:
model.to(device)

In [ ]:
probeParam()

In [ ]:
import torch.optim as optim

optimizier = optim.Adam(model.parameters(), lr=0.01) # 定义优化器和学习率
#optimizier.to(device)

In [ ]:
loss_fn = nn.MSELoss().to(device)
loss_fn

In [ ]:
%%time 
# 训练模型
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! 
# model is already to(cuda） 
num_epochs = 1000
for epoch_index in range(num_epochs):
    train_in_epoch(epoch_index)